In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

# 读取数据
data = pd.read_excel('UTS.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

# 最佳参数
best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'max_depth': 8,
    'min_child_weight': 10.0,
    'n_estimators': 113,
    'reg_alpha': 0.0,
    'reg_lambda': 1.0,
    'subsample': 1.0,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
UTS_model = xgb.XGBRegressor(**best_params)
UTS_model.fit(x_train, y_train)

# 评价模型
UTS_MSE = mean_squared_error(y_train, UTS_model.predict(x_train))
UTS_RMSE = np.sqrt(UTS_MSE)
UTS_R2 = r2_score(y_train, UTS_model.predict(x_train))
print('UTS_RMSE_train: ' + str(UTS_RMSE))
print('UTS_R2_train: ' + str(UTS_R2))

UTS_MSE_t = mean_squared_error(y_test, UTS_model.predict(x_test))
UTS_RMSE_t = np.sqrt(UTS_MSE_t)
UTS_R2_t = r2_score(y_test, UTS_model.predict(x_test))
print('UTS_RMSE_test: ' + str(UTS_RMSE_t))
print('UTS_R2_test: ' + str(UTS_R2_t))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 读取数据
data = pd.read_excel('EL.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

# 最佳参数
best_params = {
    'bootstrap': False,
    'max_depth': 10,
    'max_features': 0.5,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 55
}

# 使用最佳参数训练最终模型
EL_model = RandomForestRegressor(**best_params, random_state=25)
EL_model.fit(x_train, y_train)

# 评价模型
EL_MSE = mean_squared_error(y_train, EL_model.predict(x_train))
EL_RMSE = np.sqrt(EL_MSE)
EL_R2 = r2_score(y_train, EL_model.predict(x_train))
print('EL_RMSE_train: ' + str(EL_RMSE))
print('EL_R2_train: ' + str(EL_R2))

EL_MSE_t = mean_squared_error(y_test, EL_model.predict(x_test))
EL_RMSE_t = np.sqrt(EL_MSE_t)
EL_R2_t = r2_score(y_test, EL_model.predict(x_test))
print('EL_RMSE_test: ' + str(EL_RMSE_t))
print('EL_R2_test: ' + str(EL_R2_t))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb

# 读取数据
data = pd.read_excel('Hardness.xlsx')

# 分割数据
X = data.iloc[:, 1:]
Y = data.iloc[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=8)

# 最佳参数
best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.07141712725507017,
    'max_depth': 3,
    'min_child_weight': 1.036994365622379,
    'n_estimators': 179,
    'reg_alpha': 0.07844930740270828,
    'reg_lambda': 0.4659923785137676,
    'subsample': 0.5124079817369307,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# 使用最佳参数训练最终模型
Hardness_model = xgb.XGBRegressor(**best_params)
Hardness_model.fit(x_train, y_train)

# 评价模型
Hardness_MSE = mean_squared_error(y_train, Hardness_model.predict(x_train))
Hardness_RMSE = np.sqrt(Hardness_MSE)
Hardness_R2 = r2_score(y_train, Hardness_model.predict(x_train))
print('Hardness_RMSE_train: ' + str(Hardness_RMSE))
print('Hardness_R2_train: ' + str(Hardness_R2))

Hardness_MSE_t = mean_squared_error(y_test, Hardness_model.predict(x_test))
Hardness_RMSE_t = np.sqrt(Hardness_MSE_t)
Hardness_R2_t = r2_score(y_test, Hardness_model.predict(x_test))
print('Hardness_RMSE_test: ' + str(Hardness_RMSE_t))
print('Hardness_R2_test: ' + str(Hardness_R2_t))

In [ ]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
from openpyxl import Workbook
import os

warnings.filterwarnings("ignore")

# 目标值
target_UTS = 300  # 设定目标UTS值
target_EL = 3.5    # 设定目标EL值
target_Hardness = 130  # 设定目标Hardness值

# 权重
weight_UTS = 1  # UTS的权重
weight_EL = 1   # EL的权重
weight_Hardness = 1  # Hardness的权重

# 定义目标函数
def target_function(**params):
    params_df = pd.DataFrame([params], columns=X.columns)  # 保证特征顺序一致
    pred_UTS = UTS_model.predict(params_df)[0] + UTS_RMSE_t
    pred_EL = EL_model.predict(params_df)[0] + 2 * EL_RMSE_t
    pred_Hardness = Hardness_model.predict(params_df)[0] + Hardness_RMSE_t

    # 计算目标值：三个预测性能与目标性能的比值之和
    ratio_UTS = pred_UTS / target_UTS
    ratio_EL = pred_EL / target_EL
    ratio_Hardness = pred_Hardness / target_Hardness

    score = weight_UTS * abs(ratio_UTS - 1) + weight_EL * abs(ratio_EL - 1) + weight_Hardness * abs(ratio_Hardness - 1)

    # 将结果保存到 Excel 表
    iteration_result = {
        'Al': params.get('Al', None),
        'Cu': params.get('Cu', None),
        'Mg': params.get('Mg', None),
        'Ti': params.get('Ti', None),
        'Mn': params.get('Mn', None),
        'Si': params.get('Si', None),
        'La': params.get('La', None),
        'Ce': params.get('Ce', None),
        'pred_UTS': pred_UTS,
        'pred_EL': pred_EL,
        'pred_Hardness': pred_Hardness,
        'score': -score
    }
    save_iteration_to_excel(iteration_result)

    return -score  # 优化器会最大化目标函数，所以这里返回负值以最小化差值

# 定义特征的容许范围
pbounds = {
    'Al': (3, 8),  # 例如铝的范围从3到8
    'Cu': (0, 6),   # 例如铜的范围从0到6
    'Mg': (0.0, 0.05),   # 例如镁的范围从0到0.05
    'Ti': (0.0, 0.0),   # 例如钛的范围固定为0
    'Mn': (0.0, 0.0),   # 例如锰的范围固定为0
    'Si': (0.0, 0.0),   # 例如硅的范围固定为0
    'La': (0.0, 0.0),   # 例如镧的范围固定为0
    'Ce': (0.0, 0.0)    # 例如铈的范围固定为0
}

# 初始化 Excel 文件
def initialize_excel(file_name):
    if not os.path.exists(file_name):
        wb = Workbook()
        ws = wb.active
        ws.title = "Results"
        ws.append(['Al', 'Cu', 'Mg', 'Ti', 'Mn', 'Si', 'La', 'Ce', 'pred_UTS', 'pred_EL', 'pred_Hardness', 'score'])
        wb.save(file_name)

# 保存每次迭代的结果到 Excel 文件
def save_iteration_to_excel(result):
    file_name = "optimization_results.xlsx"
    initialize_excel(file_name)
    df = pd.DataFrame([result])
    with pd.ExcelWriter(file_name, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name="Results", index=False, header=False, startrow=writer.sheets['Results'].max_row)

# 初始化 Excel
initialize_excel("optimization_results.xlsx")

# 贝叶斯优化
optimizer = BayesianOptimization(
    f=target_function,
    pbounds=pbounds,
    random_state=1
)

# 最大迭代次数
n_iter = 300

# 运行贝叶斯优化
optimizer.maximize(init_points=50, n_iter=n_iter)

# 获取最佳参数
best_params = optimizer.max['params']
best_params_df = pd.DataFrame([best_params], columns=X.columns)  # 保证特征顺序一致
best_pred_UTS = UTS_model.predict(best_params_df)[0] + UTS_RMSE_t
best_pred_EL = EL_model.predict(best_params_df)[0] + 2 * EL_RMSE_t
best_pred_Hardness = Hardness_model.predict(best_params_df)[0] + Hardness_RMSE_t

# 容忍误差
tolerance_UTS = 20  # 可以根据需要调整
tolerance_EL = 0.5  # 可以根据需要调整
tolerance_Hardness = 10  # 可以根据需要调整

# 检查是否在容忍误差范围内
if (abs(best_pred_UTS - target_UTS) <= tolerance_UTS and
    abs(best_pred_EL - target_EL) <= tolerance_EL and
    abs(best_pred_Hardness - target_Hardness) <= tolerance_Hardness):
    print(f"找到的最佳参数: {best_params}")
    print(f"预测的UTS值: {best_pred_UTS}")
    print(f"预测的EL值: {best_pred_EL}")
    print(f"预测的Hardness值: {best_pred_Hardness}")
else:
    print("在指定迭代次数内未找到满足条件的解。")
    print("最接近的解为:")
    print(f"参数: {best_params}")
    print(f"预测的UTS值: {best_pred_UTS}")
    print(f"预测的EL值: {best_pred_EL}")
    print(f"预测的Hardness值: {best_pred_Hardness}")
    print(f"与目标值的差异: UTS差异={abs(best_pred_UTS - target_UTS)}, EL差异={abs(best_pred_EL - target_EL)}, Hardness差异={abs(best_pred_Hardness - target_Hardness)}")
